In [3]:
include("seajul.jl")

using Distributed
using PyCall
using .SJ
using Plots
using Statistics

addprocs(16)

gym = pyimport("gym")
mj = pyimport("mujoco_py")
p = pyimport("pybullet")
pyimport("pybullet_envs")
pymport

env = gym.make("HalfCheetahBulletEnv-v0")

PyObject <TimeLimit<HalfCheetahBulletEnv<HalfCheetahBulletEnv-v0>>>

In [4]:
function do_rollout(env::PyObject, W, μ, σ ; render=false)
    x = env.reset()
    done = false
    reward = 0.0
    
    X = zeros(env.observation_space.shape[1])
    U = zeros(env.action_space.shape[1])
    R = zeros(1)
    
    while !done
        
        for o in x
            push!(X,o)
        end

        
        u = clamp(W*((x-μ)./σ), env.action_space.low, env.action_space.high)
        x, r, done, _ = env.step(u)
        reward += r
        
        
        if render
            env.render()
            sleep(.02)
        end
            
        
        for a in u
            push!(U,a)
        end
        
        push!(R,r)
        
    end
    
    X = reshape(X,(env.observation_space.shape[1],:))
    U = reshape(U,(env.action_space.shape[1]),:)
     
    return X,U,R
end


do_rollout (generic function with 1 method)

In [5]:
W = randn(env.action_space.shape[1], env.observation_space.shape[1])
X,U,R = do_rollout(env, W, zeros(size(W)[2]), ones(size(W)[2]))
print(size(X))
println(findmax(X,dims=2))

(26, 1001)([0.0; 0.0; 1.0; 0.6013820171356201; 0.0; 0.36070123314857483; 0.0; 0.9281256794929504; 0.0; 0.7493743300437927; 0.8834662437438965; 1.4355136156082153; 1.101383924484253; 3.5648739337921143; 1.025519847869873; 1.9758282899856567; 1.169731616973877; 5.0; 1.2413609027862549; 4.47041130065918; 1.0; 1.0; 1.0; 1.0; 1.0; 0.0], CartesianIndex{2}[CartesianIndex(1, 1); CartesianIndex(2, 1); CartesianIndex(3, 2); CartesianIndex(4, 69); CartesianIndex(5, 1); CartesianIndex(6, 19); CartesianIndex(7, 1); CartesianIndex(8, 303); CartesianIndex(9, 1); CartesianIndex(10, 137); CartesianIndex(11, 28); CartesianIndex(12, 17); CartesianIndex(13, 343); CartesianIndex(14, 352); CartesianIndex(15, 141); CartesianIndex(16, 25); CartesianIndex(17, 21); CartesianIndex(18, 91); CartesianIndex(19, 2); CartesianIndex(20, 24); CartesianIndex(21, 26); CartesianIndex(22, 59); CartesianIndex(23, 99); CartesianIndex(24, 17); CartesianIndex(25, 43); CartesianIndex(26, 1)])


In [12]:
@everywhere include("seajul.jl")
 
W = zeros(env.action_space.shape[1], env.observation_space.shape[1])
μ = vec(zeros(env.observation_space.shape[1]))
σ = vec(std(X,dims=2))
#@time r_log, W, μ, Σ = SJ.ars!(env, W, μ, σ, num_epochs=1000, N=32,b=4,α=.02,σ=.03)
@time r_log, W = SJ.ars_v1t!(env, W, num_epochs=1000, N=32,b=4,α=.02,σ=.03)



println(W)
plot(r_log)

      From worker 45:	WARNING: replacing module SJ.
      From worker 37:	WARNING: replacing module SJ.
      From worker 38:	WARNING: replacing module SJ.
      From worker 47:	WARNING: replacing module SJ.
      From worker 41:	WARNING: replacing module SJ.
      From worker 36:	WARNING: replacing module SJ.
      From worker 39:	WARNING: replacing module SJ.
      From worker 35:	WARNING: replacing module SJ.
      From worker 42:	WARNING: replacing module SJ.
      From worker 46:	WARNING: replacing module SJ.
      From worker 49:	WARNING: replacing module SJ.
      From worker 34:	WARNING: replacing module SJ.
      From worker 48:	WARNING: replacing module SJ.
      From worker 44:	WARNING: replacing module SJ.
      From worker 40:	WARNING: replacing module SJ.
      From worker 43:	WARNING: replacing module SJ.


┌ Error: Fatal error on process 34
│   exception =
│    PyError ($(Expr(:escape, :(ccall(#= /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
│    TypeError("can't pickle traceback objects",)
│    
│    Stacktrace:
│     [1] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:60 [inlined]
│     [2] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:64 [inlined]
│     [3] _handle_error(::String) at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:81
│     [4] macro expansion at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:95 [inlined]
│     [5] #110 at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 [inlined]
│     [6] disable_sigint at ./c.jl:446 [inlined]
│     [7] __pycall! at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:42 [inlined]
│     [8] _pycall!(::PyCall.PyObject, ::Py

      From worker 34:	/home/sgillen/miniconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
      From worker 34:	  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
      From worker 34:	pybullet build time: May 20 2020 18:20:28
      From worker 35:	/home/sgillen/miniconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
      From worker 35:	  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


┌ Error: Fatal error on process 35
│   exception =
│    PyError ($(Expr(:escape, :(ccall(#= /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
│    TypeError("can't pickle traceback objects",)
│    
│    Stacktrace:
│     [1] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:60 [inlined]
│     [2] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:64 [inlined]
│     [3] _handle_error(::String) at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:81
│     [4] macro expansion at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:95 [inlined]
│     [5] #110 at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 [inlined]
│     [6] disable_sigint at ./c.jl:446 [inlined]
│     [7] __pycall! at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:42 [inlined]
│     [8] _pycall!(::PyCall.PyObject, ::Py

      From worker 35:	pybullet build time: May 20 2020 18:20:28


...

┌ Error: Fatal error on process 47
│   exception =
│    PyError ($(Expr(:escape, :(ccall(#= /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
│    TypeError("can't pickle traceback objects",)
│    
│    Stacktrace:
│     [1] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:60 [inlined]
│     [2] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:64 [inlined]
│     [3] _handle_error(::String) at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:81
│     [4] macro expansion at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:95 [inlined]
│     [5] #110 at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 [inlined]
│     [6] disable_sigint at ./c.jl:446 [inlined]
│     [7] __pycall! at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:42 [inlined]
│     [8] _pycall!(::PyCall.PyObject, ::Py

      From worker 47:	/home/sgillen/miniconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
      From worker 47:	  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
      From worker 47:	pybullet build time: May 20 2020 18:20:28
      From worker 48:	/home/sgillen/miniconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
      From worker 48:	  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
      From worker 48:	pybullet build time: May 20 2020 18:20:28


┌ Error: Fatal error on process 48
│   exception =
│    PyError ($(Expr(:escape, :(ccall(#= /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
│    TypeError("can't pickle traceback objects",)
│    
│    Stacktrace:
│     [1] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:60 [inlined]
│     [2] pyerr_check at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:64 [inlined]
│     [3] _handle_error(::String) at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:81
│     [4] macro expansion at /home/sgillen/.julia/packages/PyCall/zqDXB/src/exception.jl:95 [inlined]
│     [5] #110 at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:43 [inlined]
│     [6] disable_sigint at ./c.jl:446 [inlined]
│     [7] __pycall! at /home/sgillen/.julia/packages/PyCall/zqDXB/src/pyfncall.jl:42 [inlined]
│     [8] _pycall!(::PyCall.PyObject, ::Py

CompositeException: TaskFailedException:
ProcessExitedException(34)

...and 15 more exception(s).

Stacktrace:
 [1] sync_end(::Array{Any,1}) at ./task.jl:316
 [2] (::Distributed.var"#157#159"{Main.SJ.var"#4#7"{Float64,PyObject,SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,1},SharedArrays.SharedArray{Float64,1}},UnitRange{Int64}})() at ./task.jl:335

      From worker 49:	/home/sgillen/miniconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
      From worker 49:	  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
      From worker 49:	pybullet build time: May 20 2020 18:20:28


In [ ]:
X,U,R = do_rollout(env, W, μ, Σ, render=true)
print(sum(R))

In [ ]:
using Plots

plot(X')